In [20]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
files = glob.glob("/Users/karolinegriesbach/Downloads/*Zeitbericht*.csv")

In [4]:
df = pd.DataFrame()
for file in files:
    print(file)
    temp = pd.read_csv(file, sep=",")
    df = pd.concat([df, temp], ignore_index=True)
df

In [5]:
df["Datum"] = pd.to_datetime(df["Startdatum"], format="%d/%m/%Y")
df["Week"] = df["Datum"].dt.isocalendar().week
df[["Startdatum","Datum","Week"]]

In [6]:
df["Datum"].min(), df["Datum"].max()

In [7]:
total_hours = df["Dauer (dezimal)"].sum()
print(f"Total hours: {total_hours}")

In [8]:
pricing = df[df["Beschreibung"].str.contains("Pricing", case=False, na=False)]
pricing_hours = pricing["Dauer (dezimal)"].sum()
print(f"Total hours for Pricing: {pricing_hours}")
pricing

In [9]:
review = df[df["Beschreibung"].str.contains("Review", case=False, na=False)]
review_hours = review["Dauer (dezimal)"].sum()
print(f"Total hours for Review: {review_hours}")
review

# By Week

In [13]:
task_week = df.groupby(by=["Week","Aufgabe"])["Dauer (dezimal)"].sum().reset_index().sort_values(by="Week")
task_week

In [16]:
sns.barplot(data=task_week, x="Week", y="Dauer (dezimal)", hue="Aufgabe")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)

In [28]:
df["Beschreibung_Grouping"] = df["Beschreibung"]
df["Beschreibung_Grouping"] = np.where(df["Aufgabe"]=="CustomerSetup", "CustomerSetup",df["Beschreibung_Grouping"])
df["Beschreibung_Grouping"] = np.where(df["Aufgabe"]=="Performance Review", "Performance Review",df["Beschreibung_Grouping"])
df["Beschreibung_Grouping"] = np.where(df["Aufgabe"]=="Sonstiges", "Sonstiges",df["Beschreibung_Grouping"])
df["Beschreibung_Grouping"] = np.where(df["Aufgabe"]=="Bug Fixing", "Bug Fixing",df["Beschreibung_Grouping"])
df.groupby(by="Aufgabe")["Beschreibung_Grouping"].value_counts()

In [29]:
desc_week = df.groupby(by=["Week","Beschreibung_Grouping"])["Dauer (dezimal)"].sum().reset_index().sort_values(by="Week")
desc_week

In [35]:
df_plot

In [39]:
for week in df["Week"].sort_values().unique().tolist():
    df_plot = desc_week[desc_week["Week"] == week].sort_values(by="Beschreibung_Grouping")
    plt.title(f"Week {week}")
    sns.barplot(data=df_plot, x="Week", y="Dauer (dezimal)", hue="Beschreibung_Grouping", palette="Set2")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.show()